In [4]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# Load data (adjust path if needed)
df = pd.read_csv("data.csv", sep=';')

# Quick sanity check
df.head()


,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,17,5,171,1,1,122.0,1,19,12,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,1,15,1,9254,1,1,160.0,1,1,3,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,1,1,5,9070,1,1,122.0,1,37,37,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,1,17,2,9773,1,1,122.0,1,38,37,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,2,39,1,8014,0,1,100.0,1,37,38,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate


In [13]:
# Make sure the necessary columns exist
required_cols_viz1 = ["Unemployment rate", "Inflation rate", "GDP", "Target"]
for c in required_cols_viz1:
    if c not in df.columns:
        print(f"Missing column for viz1:", c)

targets = df["Target"].dropna().unique()

fig1 = go.Figure()
trace_meta = []  # keep track of which trace belongs to which target and type

for t in targets:
    sub = df[df["Target"] == t].copy()
    
    # Scatter points
    fig1.add_trace(
        go.Scatter(
            x=sub["Unemployment rate"],
            y=sub["Inflation rate"],
            mode="markers",
            name=f"{t} students",
            marker=dict(size=7, opacity=0.7),
            customdata=np.stack([sub["GDP"], sub["Target"]], axis=-1),
            hovertemplate=(
                "Unemployment: %{x}%<br>"
                "Inflation: %{y}%<br>"
                "GDP: %{customdata[0]}<br>"
                "Outcome: %{customdata[1]}<extra></extra>"
            )
        )
    )
    trace_meta.append(("scatter", t))
    
    # Trend line (simple linear regression of Inflation vs Unemployment)
    if len(sub) > 1:
        x = sub["Unemployment rate"].values
        y = sub["Inflation rate"].values
        m, b = np.polyfit(x, y, 1)  # y = m x + b
        x_line = np.linspace(x.min(), x.max(), 50)
        y_line = m * x_line + b
        
        fig1.add_trace(
            go.Scatter(
                x=x_line,
                y=y_line,
                mode="lines",
                name=f"{t} trend",
                line=dict(dash="dash"),
                hovertemplate=f"{t} trend line<extra></extra>"
            )
        )
        trace_meta.append(("line", t))

# Build dropdown buttons for filtering by outcome
num_traces = len(fig1.data)
buttons = []

# Button 0: show all
buttons.append(
    dict(
        label="All outcomes",
        method="update",
        args=[
            {"visible": [True] * num_traces},
            {"title": "Financial Stability vs Student Outcome (All)"}
        ]
    )
)

# One button per Target
for t in targets:
    visible = []
    for (ttype, ttarget) in trace_meta:
        visible.append(ttarget == t)
    buttons.append(
        dict(
            label=f"{t} only",
            method="update",
            args=[
                {"visible": visible},
                {"title": f"Financial Stability vs Student Outcome ({t} only)"}
            ]
        )
    )

fig1.update_layout(
    title="Financial Stability vs Student Outcome",
    xaxis_title="Unemployment rate (%)",
    yaxis_title="Inflation rate (%)",
    legend_title="Traces",
    hovermode="closest",
    updatemenus=[
        dict(
            type="dropdown",
            x=1.15,
            y=1.15,
            showactive=True,
            active=0,
            buttons=buttons
        )
    ]
)

# Add a range slider on the x-axis for fun zooming/selection
fig1.update_xaxes(rangeslider=dict(visible=True))



fig1.show()


In [12]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# Load data (change path if needed)
df = pd.read_csv("data.csv", sep=";")

targets = df["Target"].dropna().unique()

fig1 = go.Figure()
trace_meta = []   # keep track of which trace belongs to which target

for t in targets:
    sub = df[df["Target"] == t].dropna(subset=["Unemployment rate", "Inflation rate"]).copy()
    
    # --- Scatter points ---
    fig1.add_trace(
        go.Scatter(
            x=sub["Unemployment rate"],
            y=sub["Inflation rate"],
            mode="markers",
            name=f"{t} students",
            marker=dict(size=7, opacity=0.7),
        )
    )
    trace_meta.append({"kind": "scatter", "target": t})
    
    # --- Quadratic regression curve y = a x^2 + b x + c ---
    if len(sub) > 3:
        x = sub["Unemployment rate"].values
        y = sub["Inflation rate"].values
        
        # Fit a 2nd-degree polynomial (curved trend line)
        coeffs = np.polyfit(x, y, deg=2)
        
        # Smooth x-range for line
        x_line = np.linspace(x.min(), x.max(), 100)
        y_line = np.polyval(coeffs, x_line)
        
        fig1.add_trace(
            go.Scatter(
                x=x_line,
                y=y_line,
                mode="lines",
                name=f"{t} trend (quadratic)",
                line=dict(width=3, dash="dash"),
            )
        )
        trace_meta.append({"kind": "trend", "target": t})

# ---- Dropdown for selecting outcome groups ----
num_traces = len(fig1.data)
buttons = []

# Show all
buttons.append(
    dict(
        label="All outcomes",
        method="update",
        args=[{"visible": [True] * num_traces}]
    )
)

# One button per target
for t in targets:
    visible = [(meta["target"] == t) for meta in trace_meta]
    buttons.append(
        dict(
            label=f"{t} only",
            method="update",
            args=[{"visible": visible}]
        )
    )

fig1.update_layout(
    title="Financial Stability vs Student Outcome (with Quadratic Trend Lines)",
    xaxis_title="Unemployment rate (%)",
    yaxis_title="Inflation rate (%)",
    hovermode="closest",
    updatemenus=[
        dict(
            type="dropdown",
            showactive=True,
            x=1.15,
            y=1.15,
            buttons=buttons
        )
    ]
)

# Optional: add range slider on x-axis
fig1.update_xaxes(rangeslider=dict(visible=True))

fig1.show()


In [19]:
fig_fin = go.Figure()

# Density Heatmap
fig_fin.add_trace(
    go.Histogram2d(
        x=df["Mother's qualification"],
        y=df["Father's qualification"],
        colorscale="Blues",
        zsmooth="best",
        nbinsx=20,
        nbinsy=20,
        name="Student Density",
        colorbar=dict(
            title="Student Density",
            tickvals=[0, 200, 400, 600, 800, 1000, 1200],
            ticktext=["Very Low","Low","Med-Low","Medium","Med-High","High","Very High"]
        )
    )
)

# Dropout Scatter
drop = df[df["Target"] == "Dropout"]

fig_fin.add_trace(
    go.Scatter(
        x=drop["Mother's qualification"],
        y=drop["Father's qualification"],
        mode="markers",
        name="Dropout Students",
        marker=dict(color="red", size=6, opacity=0.85),
        hovertemplate="Mother Qual: %{x}<br>Father Qual: %{y}<extra></extra>"
    )
)

fig_fin.update_layout(
    title="Financial Stability Landscape (Parental Education Levels)",
    xaxis_title="Mother's Qualification Level",
    yaxis_title="Father's Qualification Level",
    hovermode="closest",
    legend=dict(x=1.02, y=1.0)
)

fig_fin.show()


In [21]:
import plotly.graph_objects as go

fig_fin = go.Figure()

# ---- Density Heatmap: all students ----
fig_fin.add_trace(
    go.Histogram2d(
        x=df["Mother's qualification"],
        y=df["Father's qualification"],
        colorscale="Blues",
        zsmooth="best",
        nbinsx=20,
        nbinsy=20,
        name="Student Density",
        colorbar=dict(
            title="Student Density",
            tickvals=[0, 200, 400, 600, 800, 1000, 1200],
            ticktext=["Very Low","Low","Med-Low","Medium","Med-High","High","Very High"]
        )
    )
)

# ---- Overlays for each outcome ----
drop = df[df["Target"] == "Dropout"]
grad = df[df["Target"] == "Graduate"]
enrl = df[df["Target"] == "Enrolled"]

fig_fin.add_trace(
    go.Scatter(
        x=drop["Mother's qualification"],
        y=drop["Father's qualification"],
        mode="markers",
        name="Dropout Students",
        marker=dict(color="red", size=6, opacity=0.85),
        hovertemplate="Outcome: Dropout<br>Mother Qual: %{x}<br>Father Qual: %{y}<extra></extra>"
    )
)


# ---- Filtering menu ----
# trace indices: 0 = heatmap, 1 = dropout, 2 = graduate, 3 = enrolled
buttons = [
    dict(
        label="All outcomes",
        method="update",
        args=[
            {"visible": [True, True, True, True]},
            {"title": "Financial Stability Landscape (All Outcomes)"}
        ]
    ),
    dict(
        label="Dropout only",
        method="update",
        args=[
            {"visible": [True, True, False, False]},
            {"title": "Financial Stability Landscape (Dropout Overlay)"}
        ]
    ),
]

fig_fin.update_layout(
    title="Financial Stability Landscape (Parental Education Levels)",
    xaxis_title="Mother's Qualification Level",
    yaxis_title="Father's Qualification Level",
    hovermode="closest",
    legend=dict(x=1.02, y=1.0),
    updatemenus=[
        dict(
            type="dropdown",
            showactive=True,
            x=1.18,
            y=1.15,
            buttons=buttons
        )
    ]
)

fig_fin.show()


This visualization maps socioeconomic status using parents’ education levels and displays only the students who eventually dropped out (red dots) over the full student density landscape. Parental education is coded numerically in the dataset, where lower values typically correspond to incomplete or basic education, and higher values represent secondary schooling, bachelor’s, master’s, and doctorate degrees. The blue density background shows where most students fall in terms of socioeconomic background, and the red dropout points reveal the specific socioeconomic status (SES) profiles associated with dropout behavior.

A clear pattern emerges: many dropouts are concentrated in the lower parental qualification region, particularly where both parents have little or no formal education. This is consistent with established research, as students from lower-SES households often face financial pressure, fewer academic resources, and weaker structural support in higher education—all of which increase dropout risk.

However, the visualization also shows a second, distinct dropout cluster in the high-qualification region, where both parents hold advanced degrees. This suggests that dropout is not solely driven by socioeconomic disadvantage. Students from highly educated families may still withdraw due to non-financial factors such as academic misalignment with their chosen program, mental health challenges, burnout, or pressure to change career paths. These students often have the safety net and flexibility to pause or restart their education without severe financial consequences.

The presence of dropout clusters at both ends of the SES spectrum implies that while financial stability and parental education strongly influence dropout likelihood, they are not the only contributing factors. Instead, dropout appears to follow multiple pathways: one driven by socioeconomic barriers, and another driven by personal, academic, or psychological factors that can affect even the most educationally advantaged students.

In [ ]:
import plotly.graph_objects as go
import numpy as np

df["SES_score"] = (df["Mother's qualification"] + df["Father's qualification"]) / 2
q1, q2 = df["SES_score"].quantile([0.33, 0.66])

ses_groups = {
    "All SES": df,
    "Low SES": df[df["SES_score"] <= q1],
    "Mid SES": df[(df["SES_score"] > q1) & (df["SES_score"] < q2)],
    "High SES": df[df["SES_score"] >= q2],
}
ses_labels = list(ses_groups.keys())

fig_fin = go.Figure()

for i, label in enumerate(ses_labels):
    sub = ses_groups[label]

    fig_fin.add_trace(
        go.Histogram2d(
            x=sub["Mother's qualification"],
            y=sub["Father's qualification"],
            colorscale="Blues",
            zsmooth="best",
            nbinsx=25,
            nbinsy=25,
            name=f"{label} - student density",
            opacity=0.65,
            colorbar=dict(
                title="Student Density"
            ),
            showscale=True if label == "All SES" else True, 
            visible=True if label == "All SES" else False
        )
    )

    sub_drop = sub[sub["Target"] == "Dropout"]
    fig_fin.add_trace(
        go.Scatter(
            x=sub_drop["Mother's qualification"],
            y=sub_drop["Father's qualification"],
            mode="markers",
            name="Dropout students (red)",
            marker=dict(color="red", size=7, opacity=0.85),
            hovertemplate="Outcome: Dropout<br>Mother qual: %{x}<br>Father qual: %{y}<extra></extra>",
            visible=True if label == "All SES" else False
        )
    )

    sub_grad = sub[sub["Target"] == "Graduate"]
    fig_fin.add_trace(
        go.Scatter(
            x=sub_grad["Mother's qualification"],
            y=sub_grad["Father's qualification"],
            mode="markers",
            name="Graduate students (green)",
            marker=dict(color="green", size=7, opacity=0.8),
            hovertemplate="Outcome: Graduate<br>Mother qual: %{x}<br>Father qual: %{y}<extra></extra>",
            visible=True if label == "All SES" else False
        )
    )

    sub_enrl = sub[sub["Target"] == "Enrolled"]
    fig_fin.add_trace(
        go.Scatter(
            x=sub_enrl["Mother's qualification"],
            y=sub_enrl["Father's qualification"],
            mode="markers",
            name="Enrolled students (orange)",
            marker=dict(color="orange", size=7, opacity=0.8),
            hovertemplate="Outcome: Enrolled<br>Mother qual: %{x}<br>Father qual: %{y}<extra></extra>",
            visible=True if label == "All SES" else False
        )
    )

n_per_group = 4
total_traces = len(fig_fin.data)
buttons = []

for i, label in enumerate(ses_labels):
    visible = [False] * total_traces
    start = i * n_per_group
    end = start + n_per_group
    for j in range(start, end):
        visible[j] = True

    buttons.append(
        dict(
            label=label,
            method="update",
            args=[
                {"visible": visible},
                {"title": f"Financial Stability Landscape ({label})"}
            ]
        )
    )

fig_fin.update_layout(
    title="Financial Stability Landscape (All SES)",
    xaxis_title="Mother's Qualification Level",
    yaxis_title="Father's Qualification Level",
    hovermode="closest",
    width=1400,
    height=650,
    margin=dict(l=70, r=150, t=120, b=70),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.03,
        xanchor="left",
        x=0,
        bgcolor="rgba(255,255,255,0.7)",
        bordercolor="lightgray",
        borderwidth=1
    ),
    updatemenus=[
        dict(
            type="dropdown",
            showactive=True,
            x=1.08,
            y=1.25,
            buttons=buttons
        )
    ]
)

fig_fin.show()


This visualization maps socioeconomic status using parents’ education levels and displays only the students who eventually dropped out (red dots) over the full student density landscape. Parental education is coded numerically in the dataset, where lower values typically correspond to incomplete or basic education, and higher values represent secondary schooling, bachelor’s, master’s, and doctorate degrees. The blue density background shows where most students fall in terms of socioeconomic background, and the red dropout points reveal the specific socioeconomic status (SES) profiles associated with dropout behavior.

A clear pattern emerges: many dropouts are concentrated in the lower parental qualification region, particularly where both parents have little or no formal education. This is consistent with established research, as students from lower-SES households often face financial pressure, fewer academic resources, and weaker structural support in higher education—all of which increase dropout risk.

However, the visualization also shows a second, distinct dropout cluster in the high-qualification region, where both parents hold advanced degrees. This suggests that dropout is not solely driven by socioeconomic disadvantage. Students from highly educated families may still withdraw due to non-financial factors such as academic misalignment with their chosen program, mental health challenges, burnout, or pressure to change career paths. These students often have the safety net and flexibility to pause or restart their education without severe financial consequences.

The presence of dropout clusters at both ends of the SES spectrum implies that while financial stability and parental education strongly influence dropout likelihood, they are not the only contributing factors. Instead, dropout appears to follow multiple pathways: one driven by socioeconomic barriers, and another driven by personal, academic, or psychological factors that can affect even the most educationally advantaged students.

In [ ]:
import numpy as np
import plotly.graph_objects as go

grade_col = "Curricular units 1st sem (grade)"
units_col = "Curricular units 1st sem (approved)"

df_acad = df.dropna(subset=[grade_col, units_col, "Target"]).copy()

grade_thr = df_acad[grade_col].median()
units_thr = df_acad[units_col].median()

hover_cols = []
for col in ["Tuition fees up to date", "Debtor", "Scholarship holder"]:
    if col in df_acad.columns:
        hover_cols.append(col)

fig_risk = go.Figure()

outcomes_order = ["Dropout", "Graduate", "Enrolled"]
outcome_colors = {
    "Dropout": "red",
    "Graduate": "green",
    "Enrolled": "orange"
}

for outcome in outcomes_order:
    color = outcome_colors[outcome]
    sub = df_acad[df_acad["Target"] == outcome]

    hover_text = []
    for _, row in sub.iterrows():
        parts = [
            f"Outcome: {outcome}",
            f"1st Sem Grade: {row[grade_col]:.2f}",
            f"Units Approved: {row[units_col]}"
        ]
        for c in hover_cols:
            parts.append(f"{c}: {row[c]}")
        hover_text.append("<br>".join(parts))

    fig_risk.add_trace(
        go.Scatter(
            x=sub[grade_col],
            y=sub[units_col],
            mode="markers",
            name=outcome,
            marker=dict(
                color=color,
                size=6,
                opacity=0.75,
                line=dict(width=0.4, color="black")
            ),
            text=hover_text,
            hovertemplate="%{text}<extra></extra>",
            visible=True  
        )
    )

for tr in fig_risk.data:
    tr.visible = (tr.name == "Dropout")

fig_risk.add_shape(
    type="line",
    x0=grade_thr, x1=grade_thr,
    y0=df_acad[units_col].min(), y1=df_acad[units_col].max(),
    line=dict(color="grey", width=2, dash="dash")
)

fig_risk.add_shape(
    type="line",
    x0=df_acad[grade_col].min(), x1=df_acad[grade_col].max(),
    y0=units_thr, y1=units_thr,
    line=dict(color="grey", width=2, dash="dash")
)

x_min, x_max = df_acad[grade_col].min(), df_acad[grade_col].max()
y_min, y_max = df_acad[units_col].min(), df_acad[units_col].max()

fig_risk.add_annotation(
    x=(x_min + grade_thr) / 2,
    y=(y_min + units_thr) / 2,
    text="Low grade<br>Low approvals<br><b>Higher dropout risk</b>",
    showarrow=False,
    font=dict(size=11, color="firebrick"),
    bgcolor="rgba(255,230,230,0.85)"
)

fig_risk.add_annotation(
    x=(x_max + grade_thr) / 2,
    y=(y_max + units_thr) / 2,
    text="High grade<br>High approvals<br><b>Lower dropout risk</b>",
    showarrow=False,
    font=dict(size=11, color="darkgreen"),
    bgcolor="rgba(230,255,230,0.85)"
)

n_traces = len(outcomes_order)  

buttons = []

buttons.append(
    dict(
        label="All outcomes",
        method="update",
        args=[
            {"visible": [True]*n_traces + [True, True]},  
            {"title": "Academic Risk Map (All Outcomes)"}
        ]
    )
)

for i, outcome in enumerate(outcomes_order):
    visible = [False]*n_traces
    visible[i] = True          
    visible += [True, True]    
    buttons.append(
        dict(
            label=f"{outcome} only",
            method="update",
            args=[
                {"visible": visible},
                {"title": f"Academic Risk Map ({outcome} Only)"}
            ]
        )
    )

fig_risk.update_layout(
    title="Academic Risk Map (Dropout Only)",
    xaxis_title="1st Semester Average Grade",
    yaxis_title="Units Approved in 1st Semester",
    hovermode="closest",
    width=1100,
    height=650,
    margin=dict(l=60, r=40, t=100, b=60),
    legend=dict(
        title="Student Outcome",
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="left",
        x=0,
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor="lightgray",
        borderwidth=1
    ),
    updatemenus=[
        dict(
            type="dropdown",
            showactive=True,
            x=1.12,
            y=1.15,
            buttons=buttons,
            active=1  
        )
    ]
)

fig_risk.update_xaxes(rangeslider=dict(visible=False))

fig_risk.show()


This visualization maps early academic performance into “risk zones” for different student outcomes. Each point represents a student, plotted by their first-semester average grade on the x-axis and the number of units they successfully completed on the y-axis. Points are color-coded by final outcome (dropout, graduate, or still enrolled). The dashed vertical and horizontal lines split the space into four quadrants using the median grade and median units approved as thresholds. The bottom-left region (low grades and low approvals) is annotated as a higher-risk zone for dropout, while the top-right region (high grades and high approvals) represents a lower-risk zone.
The dropdown menu allows us to isolate each outcome group. When viewing only dropouts, we see that many of them cluster in the low-grade, low-approval quadrant, confirming that weak first-semester performance is strongly associated with leaving the program. In contrast, graduates are more concentrated in the high-grade, high-approval quadrant, indicating strong early academic performance and successful progression. This interactive “academic risk map” makes it easy to visually identify performance profiles that are associated with higher dropout risk and those linked with persistence and graduation.